In [2]:
import sys
sys.path.append('..')

from src import device


import numpy as np
from operator import itemgetter


if device=='gpu':
    import tensorflow as tf

    def fft_multiply(A, B,u):
        # 补零
        size = 1
        while size < len(A) + len(B) - 1:
            size *= 2

        # FFT计算
        shape_a=len(A)
        shape_b=len(B)
        A = np.pad(A, (0, size - len(A)))
        B = np.pad(B, (0, size - len(B)))
        fft_A = tf.signal.fft(tf.constant(A,dtype=tf.complex64))
        fft_B = tf.signal.fft(tf.constant(B,dtype=tf.complex64))
        # 逐点相乘
        fft_result = fft_A * fft_B
        result = tf.signal.ifft(fft_result)[:shape_a+shape_b - 1]
        result = tf.math.real(result).numpy()

        # 四舍五入，处理浮点误差
        result = np.round(result).astype(int)

        return result[:u+1]

    def fft_multiply_2d(A, B, u):
        # 获取A和B的shape
        shape_A = A.shape
        shape_B = B.shape

        # 补零，确保长度为2的幂
        size_x = 1
        size_y = 1
        while size_x < shape_A[0] + shape_B[0] - 1:
            size_x *= 2
        while size_y < shape_A[1] + shape_B[1] - 1:
            size_y *= 2

        a_padded = np.pad(A, ((0, size_x - shape_A[0]), (0, size_y - shape_A[1])))
        b_padded = np.pad(B, ((0, size_x - shape_B[0]), (0, size_y - shape_B[1])))
        fft_A=tf.signal.fft2d(tf.constant(a_padded,dtype=tf.complex64))
        fft_B=tf.signal.fft2d(tf.constant(b_padded,dtype=tf.complex64))
        fft_result = fft_A * fft_B
        result = tf.math.real(tf.signal.ifft2d(fft_result))[:shape_A[0]+shape_B[0]-1,:shape_A[1]+shape_B[1]-1].numpy()

        # 四舍五入，处理浮点误差
        result = np.round(result).astype(int)

        return result[:u+1]

elif device=='cpu':
    def fft_multiply(A, B,u):
        # 补零
        size = 1
        while size < len(A) + len(B) - 1:
            size *= 2

        # FFT计算
        fft_A = np.fft.fft(A, size)
        fft_B = np.fft.fft(B, size)
        # 逐点相乘
        fft_result = fft_A * fft_B
        # IFFT计算，得到乘法结果
        result = np.fft.ifft(fft_result).real


        # 四舍五入，处理浮点误差
        result = np.round(result).astype(int)

        return result[:u+1]

    def fft_multiply_2d(A, B, u):
        # 获取A和B的shape
        shape_A = A.shape
        shape_B = B.shape

        # 补零，确保长度为2的幂
        size_x = 1
        size_y = 1
        while size_x < shape_A[0] + shape_B[0] - 1:
            size_x *= 2
        while size_y < shape_A[1] + shape_B[1] - 1:
            size_y *= 2

        # FFT计算

        fft_A = np.fft.fft2(A, s=(size_x, size_y))
        fft_B = np.fft.fft2(B, s=(size_x, size_y))

        # 逐点相乘
        fft_result = fft_A * fft_B

        # IFFT计算，得到乘法结果
        result = np.fft.ifft2(fft_result).real

        # 四舍五入，处理浮点误差
        result = np.round(result).astype(int)

        return result[:u+1]
    
def MincowskySum(A,B,u):
    
    def trans(A):
        arr=np.zeros(max(A)+1)
        arr[A]=1
        return arr
    
    A_mul_B = fft_multiply(trans(A),trans(B),u)
    
    return np.array([i for i,v in enumerate(A_mul_B) if v>0])


def MincowskySum_2d(A,B,u):
    
    def trans(S):
        c1 = np.array(list(map(itemgetter(0),S)))
        c2 = np.array(list(map(itemgetter(1),S)))
        M = np.zeros((max(c1)+1,max(c2)+1))
        M[c1,c2] = 1
        return M
    
    A_mul_B = fft_multiply_2d(trans(A),trans(B),u)
    # print(A_mul_B)

    
    return np.array(list(
        zip(*np.nonzero(np.select([A_mul_B>0], [1])))
    ))

#* example
MincowskySum_2d([(0,0),(1,1)],
                [(0,0),(2,1)],
                3
               )
MincowskySum(np.array([0,1]),np.array([0,2,4]),4)

array([0, 1, 2, 3, 4])

In [3]:
def ALLSUBSETSUM(S,u):
    # print('S,u: ',S,u)
    
    n=len(S)
    
    if n==1:
        return np.array([[0    , 0 ],
                         [S[0] , 1 ]])
    elif n==0:
        return np.array([[0    , 0 ]])
    
    Ta=S[:int(n/2)]
    Tb=S[int(n/2):]
    
    return MincowskySum_2d(ALLSUBSETSUM(Ta,u),ALLSUBSETSUM(Tb,u),u)

#* example
ALLSUBSETSUM(np.array([1,2,5]),10)

array([[0, 0],
       [1, 1],
       [2, 1],
       [3, 2],
       [5, 1],
       [6, 2],
       [7, 2],
       [8, 3]])

In [4]:
from functools import reduce


def ALLSUBSETSUMS(S,u):
    
    def split_S_by_congruence(S,b):
        Ql=[[] for i in range(b)]
        for x in S:
            y,l=np.divmod(x,b)
            Ql[l].append(y)
        return Ql
    # print('S,u: \n',S,u)
    
    n=len(S)
        
    b=int(np.floor(np.sqrt(n*np.log(n))))
    # print('b: ',b)
    # print('u/b: ',int(u/b))
    
    Ql=split_S_by_congruence(S,b)
    # print('Ql: ',Ql)

    QSubsetSumUp2u_div_by_b=list(
        map(
            lambda ql : ALLSUBSETSUM(ql,int(u/b)),
            Ql
        )
    )
    # print('QSubsetSumUpTo_u_div_by_b:\n', QSubsetSumUp2u_div_by_b)
    
    Rl=[np.array([z*b+l*j for z,j in QSubsetSumUp2u_div_by_b[l]]) for l in range(len(QSubsetSumUp2u_div_by_b))]
    # print('Rl:\n',Rl)
    
    return reduce(
                  lambda R1,R2 : MincowskySum(R1,R2,u),
                  Rl
                  )

#* example
print(ALLSUBSETSUMS(np.array([1,4,6,7,9,10]),10))


[ 0  1  4  5  6  7  8  9 10]
